In [ ]:
#|default_exp export

In [ ]:
#|export
from __future__ import annotations

# PyTest Capabilities with nbprocess

> A notebook allowing for the export of tests in an nbprocess project and thrown into a pytest-compatible file 

In [ ]:
#|export
from nbprocess.process import *
from nbprocess.read import *
from nbprocess.imports import *
from nbprocess.maker import *

from nbprocess.processors import _default_exp

In [ ]:
_test_file = "00_core.ipynb"

The goal of this notebook is to demonstrate how easy it can be to add automatic export of tests written inside of a Jupyter Notebook through `nbprocess`.

What does this look like? I'd recommend reading through [00_core](https://github.com/muellerzr/nbprocess-sandbox/blob/main/00_core.ipynb) first to get an idea, but essentially we write one big `unittest.TestCase` for each file (which is commonly done), and then we declare each of our tests through a `#|test {testname}` tag. There is also a special `#|test imports` tag for imports that should be present when performing the testing. 

Importing the exported module via `from library.{module} import *` is done automatically.

In [ ]:
#|export
from collections import defaultdict
from fastcore.foundation import L, ifnone
from execnb.nbio import *

class ExportTestProc:
    "A test proc that watches for `#|default_exp` and `#|test`"
    def __init__(self): self.tests = defaultdict(L)
    def _default_exp_(self, nbp, cell, exp_to): self.default_exp = f'test_{exp_to}'
    def _test_(self, nbp, cell, exp_to=None, nm=None, tst_cls=None): self.tests[self.default_exp].append(nbp.cell)

In [ ]:
#|export
_re_test = re.compile(r'#\|\s*test\s*$', re.MULTILINE)
_re_import = re.compile(r'#\|\s*test\s*import\s*$', re.MULTILINE)
_tab = "    "

In [ ]:
#|export
def get_directive(cell, key): 
    "Extract a top level directive from `cell`"
    return cell.directives_.get(key, None)

def _is_test_cell(cell): return cell.cell_type == "code" and nested_idx(cell, "test")

In [ ]:
#|export
def convert_pytest(cell):
    "Wraps cell contents into a pytest function"
    if _is_test_cell(cell) and not nested_idx(cell.directives_, "test", "import"):
        directives = get_directive(cell, "test")
        content = '\n'.join([f"{_tab}{c}" for c in cell.source.split("\n")])
        cell.source = f'def test_{directive[0]}(self):\n{content}'

In [ ]:
#|export
def convert_unittest(cell):
    "Wraps cell contents into a unittest test suite"
    convert_pytest(cell)
    if _is_test_cell(cell):
        if nested_idx(cell.directives_, "test", "case"):
            directives = get_directive(cell, "test")
            cell.source = f'class {cell.directives_[1]}({cell.directives_[2]})'
        elif not nested_idx(cell.directives_, "test", "import"):
            cell.source = '\n'.join([f'{_tab}{c}' for c in cell.source.split("\n")])

In [ ]:
#|export
def construct_imports(nb):
    "Generates the test imports for the notebook"
    libname = get_config().lib_name
    exp = _default_exp(nb)
    imports = ['#| test import\n', f'from {libname}.{exp} import *\n','import unittest']
    nb.cells.insert(1, mk_cell(imports))

First we can export the modules and get our `CoreTester` class, checking it has our `test_addition` function:

In [ ]:
#|export
def create_test_modules(path,dest,debug=False,mod_maker=ModuleMaker):
    exp = ExportTestProc()
    nb = NBProcessor(path, [convert_unittest, exp], preprocs=construct_imports)
    nb.process()
    is_new = True
    for mod,cells in exp.tests.items():
        mm = mod_maker(dest=dest, name=exp.default_exp, nb_path=path, is_new=is_new)
        mm.make(cells)
        is_new = False

In [ ]:
#|eval: false
create_test_modules(_test_file, "tmp")

g = exec_new("from tmp.test_core import CoreTester")
assert hasattr(g["CoreTester"], "test_addition")

Finally we can run the test case and ensure it passes:

In [ ]:
#|eval: false
import unittest
def run_case(testcase:unittest.TestCase):
    "Runs a unittest.TestCase"
    suite = unittest.defaultTestLoader.loadTestsFromTestCase(testcase)
    unittest.TextTestRunner().run(suite)

In [ ]:
#|eval: false
run_case(g["CoreTester"])

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK
